In [61]:
import numpy as np

grav = 9.806
grav = 9.806
Cp   = 1004.
Rgas = 287.04

#-----------------------------------------------------------------------------
# subroutines for grid initialization
#
def zgrid(nz, height = 20000., stag=False): 
    dz = height / float(nz)
    if stag:
        dz = height / float(nz-1)
        return np.arange(nz)*dz
    else:
        return (np.arange(nz)+0.5)*dz

#-----------------------------------------------------------------------------
#
def sounding(gz, theta_sfc = 300., qv_sfc = 0.001):
    
    #define parameters
    z_trop     = 12000.
    theta_trop = 343.
    temp_trop  = 213.
    
    # compute theta profile
    theta = np.zeros_like(gz)
    t1    = theta_sfc + (theta_trop - theta_sfc)*((gz/z_trop)**1.25)
    t2    = theta_trop * np.exp(grav * (gz - z_trop) / (Cp * temp_trop))   
    theta = np.where(gz <= z_trop, t1, t2)
    
    # compute rh profile
    rh    = np.where(gz <= z_trop, 1.0 - 0.75*((gz/z_trop)**1.25), 0.25)
    
    # integrate hydrostatic eq
    pi    = np.zeros_like(gz)
    p     = np.zeros_like(gz)
        
    if gz[0] > 1.0:
        dz = np.zeros_like(gz)
        dz[0] = 2.0*gz[0]
        for n in np.arange(1, gz.shape[0] - 1):
            dz[n] = 0.5*(gz[n+1] - gz[n-1])
    
    if gz[0] > 1.:
        pi[0] = 1.0 - 0.5*dz[0]*grav/(Cp*theta_sfc)
    
        for k in np.arange(1,gz.shape[0]):
            pi[k] = pi[k-1] - 2.0*dz[k]*grav/(Cp*(theta[k-1]+theta[k]))
    else:
        pi[0] = 1.0
        for k in np.arange(1,gz.shape[0]):
            pi[k] = pi[k-1] - 2.0*dz[k-1]*grav/(Cp*(theta[k-1]+theta[k]))
    
    # compute pressure (Pa) profile
    p = 1.0e5 * pi**(3.5088)
    t = pi*theta - 273.16
    
    # compute qv profile
    qvs = (380./p) * np.exp(17.27*((pi*theta - 273.16)/(pi*theta-36.)))
    qv  = qvs*rh
    qv  = np.where( qv > qv_sfc/1000., qv_sfc/1000., qv)
    
    # recompute RH so that it reflects the new qv_sfc limit - needed to dewpoint computation.
    rh  = qv/qvs
    
    ess = 6.112 * np.exp(17.67 * t / (t + 243.5))
    val = np.log(rh * ess/6.112)
    td  = 243.5 * val / (17.67 - val)
    
    den = p / (t+273.16 * Rgas)
    
    return pi, theta, qv, p, t, td, den

#-----------------------------------------------------------------------------
#
def shear(gz, type=1, shear=12.5, depth=2500.):

    if type == 0.0:
        return np.zeros_like(gz), np.zeros_like(gz)

    if type == 1:  # 1D WK squall line or supercell shear...

        scale = gz / depth

        u = np.where(scale <= 1.0, shear*scale, shear)

        return u, np.zeros_like(gz)

    if type == 2:  # 2D WK squall line or supercell shear...

        depth = 5000.
        scale = gz / depth
        ugnd  = shear / np.pi
        angle = np.deg2rad(180.0 - (1.0 - scale))
        u     = np.where(scale <= 1.0, ugnd*np.cos(angle), ugnd)
        v     = np.where(scale <= 1.0, ugnd*np.sin(angle),  0.0)

        return u, v

    if type == 3:  # Quarter circle shear

        depth = 3000.
        scale = gz / depth
        vmag  = 15.0

        angle = np.deg2rad(180.0 - 90.0*scale)

        u     = np.where(scale <= 1.0, vmag*np.cos(angle), 0.006*(gz-3000.) )
        v     = np.where(scale <= 1.0, vmag*np.sin(angle), vmag)

        return np.where(u <= 40., u, 40.), v
        
#-----------------------------------------------------------------------------
#
def write_2_file(p0, th0, q0, z, theta, qv, u, v, filename='wk.sounding'):
    
    print(filename)
    
    with open(filename, 'w') as f:
        f.write("%12.4f %12.4f  %12.4f \n" % (p0, th0, q0))
        for k in np.arange(z.shape[0]):
            f.write("%12.4f  %12.4f  %12.4f  %12.4f  %12.4f\n" % (z[k], theta[k], 1000.*qv[k], u[k], v[k]))
        
    f.close()


In [58]:
#
# Parameters for squall line soundings for FV3 tests
#

# Create grid
gze = zgrid(301, height = 30000., stag=True)
gzc = zgrid(300, height = 30000.)
dz  = gze[1:] - gze[:-1]

# create soundings

shear0 = 12.5

u, v = shear(gze, shear = 0, type=1)

qv0 = [11.0,12.0,13.0,14.0,15.0,16.0]

for q in qv0:
    
    pi, theta, qv, p, t, td, den = sounding(gze, dz, theta_sfc=300.5, qv_sfc=q)
    
    write_2_file(1000., 300.5, q, gze, theta, qv, u, v, filename = "input_sounding_qv%2.2i_us%4.1f" % (q, shear0))


TypeError: sounding() got multiple values for argument 'theta_sfc'

In [47]:
#
# Parameters for warm thermal with 500 mb tropopause.
#


gze = zgrid(301, height = 30000., stag=True)
gzc = zgrid(300, height = 30000.)
dz  = gze[1:] - gze[:-1]

# create soundings

shear0 = 0.0

u, v = shear(gze, shear = shear0, type=0)

pi, theta, qv, p, t, td, den = sounding(gze, dz, theta_sfc=300.)

write_2_file(1000., 300., 0.001, gze, theta, qv, u, v, filename = "bubble.sound" )



TypeError: sounding() got multiple values for argument 'theta_sfc'

In [72]:
# read in sounding heights for WRF

# path = "/work/wicker/Odin_scr/cm1r20.3/run/Soundings/input_grid_C2500"

# with open(path, 'r') as f:
#     strings = f.readlines()

zh = [0.01073497, 0.04024141, 0.08258476, 0.1361145, 0.2023856, 0.2813303,
    0.372868, 0.4769054, 0.5933367, 0.7220436, 0.8628909, 1.015673, 1.180161,
    1.356162, 1.543466, 1.740841, 1.945226, 2.154314, 2.368158, 2.587233,
    2.811645, 3.0415, 3.276904, 3.517966, 3.764791, 4.017487, 4.276159,
    4.540912, 4.811849, 5.08907, 5.372672, 5.662747, 5.959383, 6.262661,
    6.572655, 6.889427, 7.213032, 7.543508, 7.880885, 8.225174, 8.576372,
    8.934463, 9.299411, 9.671172, 10.0497, 10.43492, 10.82679, 11.22524,
    11.63023, 12.0424, 12.46476, 12.90181, 13.36197, 13.86196, 14.42729,
    15.09925, 15.95271, 17.11467, 18.66372, 20.95429]
    
zhgts = np.asarray([1000.*float(i) for i in zh])

zhgts = zgrid(300, height = 30000.)

for n in np.arange(len(u)):
    print(zhgts[n])

pi, theta, qv, p, t, td, den  = sounding(zhgts, theta_sfc = 300., qv_sfc = 14.0)

u, v = shear(zhgts, type=3)

# for n in np.arange(len(u)):
#     print( n, u[n], v[n] )

    
write_2_file(1000., 300., 16., zhgts, theta, qv, u, v, filename='wk14.sounding')

50.0
150.0
250.0
350.0
450.0
550.0
650.0
750.0
850.0
950.0
1050.0
1150.0
1250.0
1350.0
1450.0
1550.0
1650.0
1750.0
1850.0
1950.0
2050.0
2150.0
2250.0
2350.0
2450.0
2550.0
2650.0
2750.0
2850.0
2950.0
3050.0
3150.0
3250.0
3350.0
3450.0
3550.0
3650.0
3750.0
3850.0
3950.0
4050.0
4150.0
4250.0
4350.0
4450.0
4550.0
4650.0
4750.0
4850.0
4950.0
5050.0
5150.0
5250.0
5350.0
5450.0
5550.0
5650.0
5750.0
5850.0
5950.0
6050.0
6150.0
6250.0
6350.0
6450.0
6550.0
6650.0
6750.0
6850.0
6950.0
7050.0
7150.0
7250.0
7350.0
7450.0
7550.0
7650.0
7750.0
7850.0
7950.0
8050.0
8150.0
8250.0
8350.0
8450.0
8550.0
8650.0
8750.0
8850.0
8950.0
9050.0
9150.0
9250.0
9350.0
9450.0
9550.0
9650.0
9750.0
9850.0
9950.0
10050.0
10150.0
10250.0
10350.0
10450.0
10550.0
10650.0
10750.0
10850.0
10950.0
11050.0
11150.0
11250.0
11350.0
11450.0
11550.0
11650.0
11750.0
11850.0
11950.0
12050.0
12150.0
12250.0
12350.0
12450.0
12550.0
12650.0
12750.0
12850.0
12950.0
13050.0
13150.0
13250.0
13350.0
13450.0
13550.0
13650.0
13750.0
13850.0